In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

# Graph Traversal

In [ ]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

In [ ]:
import bw_graph_tools as bgt

In [ ]:
bgt.GraphTraversalSettings?

In [ ]:
settings = bgt.GraphTraversalSettings(max_calc=500)

In [ ]:
node = bd.get_node(name="market for photovoltaic laminate, CIS", database="ecoinvent-3.12-cutoff")

In [ ]:
random_ic = (
    'ecoinvent-3.12',
    'ReCiPe 2016 v1.03, endpoint (I) no LT',
    'total: human health no LT',
    'human health no LT'
)

In [ ]:
fu, data_objs, _ = bd.prepare_lca_inputs({node: 1}, method=random_ic, remapping=False)

In [ ]:
lca = bc.LCA(fu, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

In [ ]:
gt = bgt.NewNodeEachVisitGraphTraversal(
    lca=lca,
    settings=settings,
)

In [ ]:
gt.traverse()

In [ ]:
len(gt.nodes), len(gt.edges), len(gt.flows)

# Small fixes

## Message when deleting project

In [ ]:
bd.projects.delete_project("Bicycle example")

In [ ]:
bd.projects.set_current("Bicycle example")

In [ ]:
bd.databases

## Edge display string

In [ ]:
cf = bd.get_node(database='🚲', name='carbon fibre production')
for edge in cf.exchanges():
    print(edge)

# Extracting a subgraph from ecoinvent to a new database

In [ ]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

In [ ]:
db = bd.Database("ecoinvent-3.12-cutoff")

In [ ]:
laminate = bd.get_node(name="market for photovoltaic laminate, CIS", database="ecoinvent-3.12-cutoff")
ba.print_recursive_supply_chain(laminate, max_level=2)

In [ ]:
panel = bd.get_node(name="market for photovoltaic panel, CIS", database="ecoinvent-3.12-cutoff")
ba.print_recursive_supply_chain(panel, max_level=1)

In [ ]:
nodes = [laminate, panel] + [exc.input for exc in laminate.technosphere()] + [exc.input for exc in panel.technosphere()]
nodes

In [ ]:
subgraph = bd.Database("PV production")
subgraph.register()

In [ ]:
new_nodes = db.copy_activities(nodes, "PV production")

In [ ]:
list(subgraph)

In [ ]:
new_laminate = bd.get_node(name='market for photovoltaic laminate, CIS', database="PV production")

In [ ]:
for exc in new_laminate.technosphere():
    print(exc.input["database"], exc)

# Creating aggregated processes from unit processes

We can do this by calculating the inventory and reading the non-zero elementary flow amounts.

In [ ]:
agg, _ = new_laminate.create_aggregated_process(database="PV production", name="CIS laminate, aggregated")

In [ ]:
len(agg.biosphere()), len(agg.technosphere()), len(agg.production())

In [ ]:
gt.flows